In [10]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from IPython import display;

sample_rate = 44100  # Sampling rate
duration = 2.0       # Signal duration

t = np.linspace(0, duration, int(sample_rate * duration))

frequency = 440  # Signal frequency
sine_wave = 0.5 * np.sin(2 * np.pi * frequency * t)
duty_cycle = 0.5  # Fraction of high value
pulse_wave = 0.5 * (np.sign(np.sin(2 * np.pi * frequency * t) - np.cos(duty_cycle * np.pi)) + 1)
triangular_wave = 2 * np.abs(2 * (t * frequency - np.floor(t * frequency + 0.5))) - 1
sawtooth_wave = 2 * (t * frequency - np.floor(t * frequency + 0.5))
noise = np.random.normal(0, 0.1, t.shape)

write('sineWave.wav', sample_rate, sine_wave.astype(np.float32))
write('pulseWave.wav', sample_rate, pulse_wave.astype(np.float32))
write('triangularWave.wav', sample_rate, triangular_wave.astype(np.float32))
write('sawtoothWave.wav', sample_rate, sawtooth_wave.astype(np.float32))
write('noise.wav', sample_rate, noise.astype(np.float32))

In [32]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

sample_rate = 44100  # Sample rate
duration = 2.0      # Signal duration

t = np.linspace(0, duration, int(sample_rate * duration))

frequency = 440
noise = np.random.normal(0, 0.5, t.shape)
duty_cycle = 0.2  # Fraction of high value
pulse_wave = 0.5 * (np.sign(np.sin(2 * np.pi * frequency * t) - np.cos(duty_cycle * np.pi)) + 1)

polyphonic_wave = noise + pulse_wave

polyphonic_wave1_normalized = polyphonic_wave / np.max(np.abs(polyphonic_wave))

write('noisePulseWave.wav', sample_rate, polyphonic_wave1_normalized.astype(np.float32))

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io.wavfile import write

sample_rate = 44100   # Sample rate
duration = 2.0       # Signal duration
t = np.linspace(0, duration, int(sample_rate * duration))

carrier_freq = 440 # Carrier signal frequency
carrier_wave = 0.5 * np.abs(2 * (t * carrier_freq - np.floor(t * carrier_freq + 0.5))) - 1 #Triangle wave

# Modulating signal
mod_freq = 2
mod_sawtooth = 0.5 * (t * mod_freq - np.floor(t * mod_freq + 0.5)) #Sawtooth wave

am_wave_sawtooth = (1 + mod_sawtooth) * carrier_wave

write('amTriangleCarrierSawtoothMod.wav', sample_rate, am_wave_sawtooth.astype(np.float32))

In [35]:
import enum
import wave
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import pareto


class Signals(enum.Enum):
    SIN = 0,
    TRIANGLE = 1,
    PULSE = 2
    SAWTOOTH = 3,
    WHITE_NOISE = 4


class SoundSignal:
    data = np.array([])
    name = ''

    def __init__(self, a, fr, n, phi):
        self.a = a
        self.fr = fr
        self.n = n
        self.phi = phi

    def generate_sound(self):
        raise NotImplementedError("Subclasses must implement this method")

    def freq_sound(self, modulate):
        raise NotImplementedError("Subclasses must implement this method")

    def to_wav(self):
        max_amplitude = np.iinfo(np.int16).max
        audio_data = (self.data * max_amplitude).astype(np.int16)

        with wave.open(self.name + '.wav', 'w') as wav_file:
            wav_file.setnchannels(1)
            wav_file.setsampwidth(2)
            wav_file.setframerate(44100)
            wav_file.writeframes(audio_data.tobytes())

    def ampl_sound(self, modulate):
        raise NotImplementedError("Subclasses must implement this method")

    def plot_signal(self):
        raise NotImplementedError("Subclasses must implement this method")


class SinSignal(SoundSignal):
    name = 'sin'

    def generate_sound(self):
        n0 = np.arange(self.n, dtype=float)
        self.data = self.a * np.sin(2 * np.pi * self.fr * n0 / 44100 + self.phi)
        return self.data

    def freq_sound(self, modulate):
        modulate.generate_sound()

        self.data = np.arange(self.n) / self.n
        phi = self.phi
        for i in range(0, self.n):
            self.data[i] = np.sin(phi) * self.a
            phi += 2 * np.pi * self.fr * (1 + modulate.data[i]) / 44100


    def ampl_sound(self, modulate):
        modulate.generate_sound()
        self.data = np.arange(self.n) / self.n
        for i in range(0, self.n):
            self.data[i] = self.a * (1 + modulate.data[i]) * np.sin(2 * np.pi * self.fr * i / 44100 + self.phi)


    def plot_signal(self):
        n0 = np.arange(self.n)
        plt.figure(figsize=(10, 4))
        plt.plot(n0, self.data, label='Синусоидальный сигнал')
        plt.title('Синусоидальный сигнал')
        plt.xlabel('Выборки')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()
        plt.show()


class TriangleSignal(SoundSignal):
    name = 'triangle'

    def generate_sound(self):

        n = np.arange(self.n)
        self.data = (2 * self.a / np.pi) * (
                np.abs((2 * np.pi * self.fr * n / 44100 + self.phi + 3 * np.pi / 2) % (2 * np.pi) - np.pi) - (
                np.pi / 2))
        return self.data

    def ampl_sound(self, modulate):
        modulate.generate_sound()

        self.data = np.arange(self.n)
        for i in range(0, self.n):
            self.data[i] = (2 * self.a * (1 + modulate.data[i]) / np.pi) * (
                    np.abs((2 * np.pi * self.fr * i / 44100 + self.phi + 3 * np.pi / 2) % (2 * np.pi) - np.pi) - (
                    np.pi / 2))

    def freq_sound(self, modulate):
        modulate.generate_sound()

        self.data = np.arange(self.n) / self.n
        phi = self.phi
        for i in range(0, self.n):
            self.data[i] = (2 * self.a * (1 + modulate.data[i]) / np.pi) * (
                    np.abs((phi + 3 * np.pi / 2) % (2 * np.pi) - np.pi) - (
                    np.pi / 2))
            phi += 2 * np.pi * self.fr * (1 + modulate.data[i]) / self.n

    def plot_signal(self):
        n = np.arange(self.n)
        plt.figure(figsize=(10, 4))
        plt.plot(n, self.data, label='Треугольный сигнал')
        plt.title('Треугольный сигнал')
        plt.xlabel('Выборки')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()
        plt.show()


class PulseSignal(SoundSignal):
    name = 'pulse'

    def __init__(self, a, fr, n, phi, d=0.5):
        super(PulseSignal, self).__init__(a, fr, n, phi)
        self.d = d

    def generate_sound(self):
        n = np.arange(self.n)
        mod_value = (2 * np.pi * self.fr * n / 44100 + self.phi) % (2 * np.pi)
        self.data = np.where(mod_value / (2 * np.pi) <= self.d, self.a, -self.a)
        return self.data

    def ampl_sound(self, modulate):
        modulate.generate_sound()

        mod_value = np.arange(self.n)
        for i in range(0, self.n):
            mod_value[i] = (1 + modulate.data[i]) * (2 * np.pi * self.fr * i / 44100 + self.phi) % (2 * np.pi)

        self.data = np.where(mod_value / (2 * np.pi) <= self.d, self.a, -self.a)

    def freq_sound(self, modulate):
        modulate.generate_sound()

        self.data = np.arange(self.n) / self.n
        phi = self.phi
        for i in range(0, self.n):
            mod_value = phi % (2 * np.pi)
            phi += 2 * np.pi * self.fr * (1 + modulate.data[i]) / 44100

            self.data[i] = np.where(mod_value / (2 * np.pi) <= self.d, self.a, -self.a)




    def plot_signal(self):
        n = np.arange(self.n)
        plt.figure(figsize=(10, 4))
        plt.plot(n, self.data, label='Импульсный сигнал')
        plt.title('Импульсный сигнал')
        plt.xlabel('Выборки')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()
        plt.show()


class Sawtooth(SoundSignal):
    name = 'sawtooth'

    def generate_sound(self):
        n = np.arange(self.n)
        self.data = (self.a / np.pi) * ((2 * np.pi * self.fr * n / 44100 + self.phi + np.pi) % (2 * np.pi) - np.pi)
        return self.data

    def ampl_sound(self, modulate):
        modulate.generate_sound()

        self.data = np.arange(self.n) / self.n

        for i in range(0, self.n):
            self.data[i] = (self.a * (1 + modulate.data[i]) / np.pi) * ((2 * np.pi * self.fr * i / 44100 + self.phi + np.pi) % (2 * np.pi) - np.pi)

    def freq_sound(self, modulate):
        modulate.generate_sound()

        self.data = np.arange(self.n) / self.n
        phi = self.phi
        for i in range(0, self.n):
            self.data[i] = (self.a / np.pi) * ((phi + np.pi) % (2 * np.pi) - np.pi)
            phi += 2 * np.pi * self.fr * (1 + modulate.data[i])/ 44100


    def plot_signal(self):
        n = np.arange(self.n)
        plt.figure(figsize=(10, 4))
        plt.plot(n, self.data, label='Зубчатый сигнал')
        plt.title('Зубчатый сигнал')
        plt.xlabel('Выборки')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.ylim(-self.a * 1.1, self.a * 1.1)
        plt.legend()
        plt.show()


class WhiteNoise(SoundSignal):
    name = 'white_noise'

    def __init__(self, duration=1, sample_rate=44100):
        super().__init__(a=1, fr=0, n=int(sample_rate * duration), phi=0)
        self.duration = duration


    def generate_sound(self):
        self.data = np.random.uniform(-1, 1, self.n)
        return self.data

    def plot_signal(self):
        n = np.arange(self.n)
        plt.figure(figsize=(10, 4))
        plt.plot(n, self.data, label='Белый шум', marker='o', markersize=1)
        plt.title('Белый шум')
        plt.xlabel('Выборки')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.ylim(-1.1, 1.1)
        plt.legend()
        plt.show()


class PolyphonicSignal(SoundSignal):
    def __init__(self, signals, n):
        super().__init__(a=0, fr=0, n=0, phi=0)
        self.signals = signals
        self.n = n
        self.name = 'polyphonic'

    def generate_sound(self):
        self.data = np.zeros(self.n, dtype=float)
        for signal in self.signals:
            self.data += signal.data

        return self.data

    def plot_signal(self):
        n = np.arange(self.n)
        plt.figure(figsize=(10, 4))
        plt.plot(n, self.data, label='Полифонический сигнал')
        plt.title('Полифонический сигнал')
        plt.xlabel('Выборки')
        plt.ylabel('Амплитуда')
        plt.grid()
        plt.legend()
        plt.show()

n = 44100
phi = 0

mod = PulseSignal(0.5, 1, n * 5, phi)
carr = PulseSignal(0.5, 440, n * 5, phi)

carr.freq_sound(mod)
carr.name = carr.name + '_freq_mod'
carr.to_wav()